In [ ]:
# Librerias utilizadas

import requests
from PIL import Image, ImageOps,ImageFilter
import pytesseract
import pandas as pd
from os.path import join, exists
import cv2
from matplotlib import pyplot as plt
import re

In [1]:
import lotto

INSERT INTO SORTEOS ( ID, FECHA ) VALUES ( :id, :fecha )
INSERT INTO PREMIADOS ( ID_SORTEO, ORDEN, PREMIADO ) VALUES ( :id_sorteo, :orden, :premiado )
INSERT INTO PROBABILIDADES ( POSICION, DIGITO, PROBABILIDAD, RANKING ) VALUES ( :posicion, :digito, :probabilidad, :ranking )


In [ ]:
lotto.inicializar()

In [ ]:
lotto.transaccionar(lotto.poblar_probabilidades, {})

In [ ]:
# Paths relativos
PIC_FOLDER = 'lotto_pics'
URL_BASE = "https://ventas-api.loteria.com.ec/uploads/boletines/T{0}.jpg"

In [ ]:
# Ultimo Sorteo
ULTIMO_SORTEO = 23053

In [ ]:
# Extrae ultimas Imagenes
sorteo = ULTIMO_SORTEO
while True:
    path_archivo = join(PIC_FOLDER,"{0}.jpg".format(sorteo))
    if exists(path_archivo):
        print('Boletin {0} ya existe'.format(sorteo))
        break
    respuesta = requests.get(URL_BASE.format(sorteo), verify = False)
    if respuesta.status_code != 200:        
        respuesta.close()
        print('Boletin {0} ya no esta disponible'.format(sorteo))
        break
    archivo = open( path_archivo, "wb" )
    archivo.write(respuesta.content)
    archivo.close()
    print('Boletin {0} extraido'.format(sorteo))
    sorteo -= 1
print('Secuencia finalizada')

In [ ]:
def  extraer_numeros_loteria(numero_sorteo, path_archivo, umbral = 242, mostrar_imagen=False, solo_numeros_loteria = True):
    imagen = cv2.imread(path_archivo, cv2.IMREAD_GRAYSCALE)
    imagen_umbralizada = imagen
    ret, imagen_umbralizada = cv2.threshold(imagen, umbral, 255, cv2.THRESH_BINARY)    
    tamano = imagen_umbralizada.shape[:2]
    imagen_umbralizada = imagen_umbralizada[ : int(tamano[0] * 0.55) ,  :  ]
    
    hallazgos = pytesseract.image_to_string( imagen_umbralizada ).split()
    expresion_num_loteria = re.compile(r'^\d{6}$')
    if solo_numeros_loteria:
        hallazgos = [ x for x in hallazgos if expresion_num_loteria.match(x) ]
    hallazgos1 = []
    sorteo = {'id': numero_sorteo, 'fecha': ''}
    lotto.transaccionar(lotto.crear_sorteo, sorteo)
    orden = 1
    for h in hallazgos:
        if h not in hallazgos1:
            premiado = {'id_sorteo': numero_sorteo, 'orden':orden, 'premiado': h}
            lotto.transaccionar(lotto.crear_premiado, premiado)
            hallazgos1.append(h)
            orden += 1
    if mostrar_imagen:
        print(imagen_umbralizada.shape)
        plt.figure(figsize=(10,10))
        plt.imshow(cv2.cvtColor(imagen_umbralizada, cv2.COLOR_GRAY2BGR))
        plt.show()
        plt.close()
    
    return hallazgos1


In [ ]:
# Persiste los sorteos desde imagenes

sorteo = ULTIMO_SORTEO
umbrales = [242,128,64]
while True:
    path_archivo = join(PIC_FOLDER,"{0}.jpg".format(sorteo))
    if not exists(path_archivo):
        print("info para sorteo {0} no disponible".format(sorteo))
        break
    numeros_encontrados = []
    for umbral in umbrales:
        numeros_encontrados1 = extraer_numeros_loteria ( sorteo, path_archivo , umbral= 242 )
        if len(numeros_encontrados1) > len(numeros_encontrados):
            numeros_encontrados = numeros_encontrados1
    linea = "{0},{1}".format(sorteo,','.join(numeros_encontrados))
    # archivo = open(path_data,'at')
    # print(linea, file=archivo)
    print(linea)
    # archivo.close()
    sorteo -= 1
print("Secuencia terminada")

In [2]:
lotto.calcular_probabilidades()

In [3]:
lotto.mostrar_probabilidad('000000')

['danger', 'success', 'danger', 'success', 'success', 'success']

In [ ]:
archivo.close()

In [ ]:
# Se cara sorteos como dataframe
from nltk.probability import FreqDist

path_data = "salida.txt"
sorteos = []
with open(path_data, 'rt') as archivo:    
    for linea in archivo:
        arreglo_linea = linea[:-1].split(',')
        tamano_arreglo = len(arreglo_linea) - 1
        if tamano_arreglo < 2:
            continue
        favorecidos = arreglo_linea[1:]
        print(favorecidos)
        digitos = [ [x[pos] for x in favorecidos ] for pos in range(0,6) ]
        distribuciones = [ str(dict(FreqDist(pos))) for pos in digitos ]
        registro = {'SORTEO': int(arreglo_linea[0]) , 'FAVORECIDOS': ','.join(favorecidos), 'CONTEO FAVORECIDOS': tamano_arreglo }
        for dpos in range(0,6):
            registro['D{0}'.format(dpos+1)] = distribuciones[dpos]
        sorteos.append(registro)

sorteos = pd.DataFrame(sorteos)
sorteos


In [ ]:
# Conteo de Sorteos por conteo favorecidos\
sorteos[['SORTEO','CONTEO FAVORECIDOS']].groupby(['CONTEO FAVORECIDOS']).count()

In [ ]:
sorteos.loc[sorteos['CONTEO FAVORECIDOS'] == 17]

In [ ]:
from nltk.probability import FreqDist

linea = '22895,906085,124822,981826,998854,347723,777305,553624,774288,903696,127377,375818,936178,787748,156995,472867,275541'
arreglo_linea = linea.split(',')
tamano_arreglo = len(arreglo_linea) - 1
favorecidos = arreglo_linea[1:]
digitos = [ [x[pos] for x in favorecidos ] for pos in range(0,6) ]
distribuciones = [ str(dict(FreqDist(pos))) for pos in digitos ]
distribuciones

In [ ]:
#import nltk
#nltk.download('stopwords')

from pyresparser import ResumeParser

data = ResumeParser('C:\\Users\\XAVIER\\Documents\\RESUME-Marcelo Merizalde.pdf').get_extracted_data()

In [ ]:
# Persistiendo 
chr(65)